<a href="https://colab.research.google.com/github/ravi-prakash1907/Machine-Learning-for-Cyber-Security/blob/main/Labs/deepNN_lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment-3 :  Diabetes Onset Prediction using Deep-Learning

In [1]:
## lib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn import preprocessing
from tensorflow import keras
from keras.models import Sequential 
from keras.layers import Dense

import requests

## Data Collection

In [2]:
## pulling dataset from my GitHub

In [3]:
def downloadCSV(fileURL, saveAs='downloaded.csv'):
  req = requests.get(fileURL)
  fileURLContent = req.content
  csv_file = open(saveAs, 'wb')
  
  csv_file.write(fileURLContent)
  csv_file.close()

In [9]:
dataLoc = "https://raw.githubusercontent.com/ravi-prakash1907/Machine-Learning-for-Cyber-Security/main/Labs/dataset/diabetes.csv"

downloadCSV(dataLoc,'diabetes.csv')

In [10]:
df = pd.read_csv("diabetes.csv")

## Data Understanding

In [11]:
df.head()

,children,Glucose,BP,skin thickness,insulin,BMI,pedigree,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [12]:
## looking at the shape
df.shape

(768, 9)

In [13]:
## looking for the detailed description of the dataset
df.describe()

,children,Glucose,BP,skin thickness,insulin,BMI,pedigree,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


**Hence, our aim is to predict the value of last column i.e. `class`**

## Data Preparation

In [42]:
## train - test split : 70:30
X_train, X_test, Y_train, Y_test = train_test_split(df[df.columns[:-1]], df['class'], test_size=0.3)

In [43]:
## splitting the test set further in : test & validation set
X_valid, X_test, Y_valid, Y_test = train_test_split(X_test, Y_test, test_size=0.5)

In [44]:
## here traing data has 70%, while validation and testing have 15%-15% of the whole dataset

### Train-Test-Validation Data

In [45]:
# features
display(X_train.head(3))
display(X_test.head(3))
display(X_valid.head(3))

,children,Glucose,BP,skin thickness,insulin,BMI,pedigree,age
738,2,99,60,17,160,36.6,0.453,21
383,1,90,62,18,59,25.1,1.268,25
693,7,129,68,49,125,38.5,0.439,43


,children,Glucose,BP,skin thickness,insulin,BMI,pedigree,age
496,5,110,68,0,0,26.0,0.292,30
607,1,92,62,25,41,19.5,0.482,25
237,0,179,90,27,0,44.1,0.686,23


,children,Glucose,BP,skin thickness,insulin,BMI,pedigree,age
210,2,81,60,22,0,27.7,0.290,25
34,10,122,78,31,0,27.6,0.512,45
278,5,114,74,0,0,24.9,0.744,57


In [46]:
# labels
display(Y_train.head(3))
display(Y_test.head(3))
display(Y_valid.head(3))

738    0
383    0
693    1
Name: class, dtype: int64

496    0
607    0
237    1
Name: class, dtype: int64

210    0
34     0
278    0
Name: class, dtype: int64

In [47]:
## normalization
min_max_scaler = preprocessing.MinMaxScaler()

X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)
X_valid = min_max_scaler.fit_transform(X_valid)

## Model Build

In [48]:
model = Sequential([
                    Dense(32, activation='relu', input_shape=(8,)),
                    Dense(32, activation='relu'),
                    Dense(32, activation='relu'),
                    Dense(1, activation='sigmoid')
])

In [49]:
model.compile(optimizer='sgd',
                loss='binary_crossentropy',
                metrics=['accuracy', 'mse'])

In [50]:

class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 150 == 0: print('')
    print('.',end='')


early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=200)
history = model.fit(X_train, Y_train, epochs=1200, verbose=0, validation_data=(X_valid, Y_valid),# validation_split=0.1,
                    callbacks = [early_stop, PrintDot()] )
  
hist = pd.DataFrame(history.history)
hist['epoc'] = history.epoch

rmse_final = np.sqrt(float(hist['val_mse'].tail(1)))
print()
print("Final RMSE on val: ",round(rmse_final,3))
hist.iloc[-1:]


......................................................................................................................................................
......................................................................................................................................................
......................................................................................................................................................
......................................................................................................................................................
......................................................................................................................................................
......................................................................................................................................................
.............................................................................................

,loss,accuracy,mse,val_loss,val_accuracy,val_mse,epoc
1069,0.439195,0.787709,0.143101,0.405905,0.8,0.132981,1069


### Evaluation

In [51]:
mse, acc, _ = model.evaluate(X_test, Y_test)
rmse = np.sqrt(mse)
print("RMSE on test: ",round(rmse,3))
print("Accuracy on test: ",round(acc,3))

4/4 [==============================] - 0s 2ms/step - loss: 0.3911 - accuracy: 0.7931 - mse: 0.1262
RMSE on test:  0.625
Accuracy on test:  0.793


### Predictions  
The model can predict with an  accuracy of **79.3%**

In [121]:
x = X_test
y = list(Y_test)

In [122]:
predicted = model.predict(x)
predicted = predicted.round().reshape(116,)
predicted = list(predicted.astype('int'))

In [124]:
print("Actual values of initial 10 elments of test set:    ",y[:10])
print("Predicted values of initial 10 elments of test set: ",predicted[:10])

Actual values of initial 10 elments of test set:     [0, 0, 1, 1, 0, 0, 0, 0, 0, 1]
Predicted values of initial 10 elments of test set:  [0, 0, 1, 1, 0, 0, 0, 0, 1, 1]
